<a href="https://colab.research.google.com/github/raduButucelea23/HF_Transformers/blob/main/course/en/chapter3/section2_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing the data (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

# Preparing the env

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

# Loading the dataset

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "sst2")
raw_datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [3]:
raw_train_dataset = raw_datasets["train"]

In [4]:
raw_train_dataset.features



{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

# Sampling 2 models

Choosing models && tokenizer

In [5]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# import torch.nn.functional as F

# untrn_checkpoint = "bert-base-uncased"
# trn_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

# untrn_tokenizer = AutoTokenizer.from_pretrained(untrn_checkpoint)
# trn_tokenizer = AutoTokenizer.from_pretrained(trn_checkpoint)

# untrn_model = AutoModelForSequenceClassification.from_pretrained(untrn_checkpoint)
# trn_model = AutoModelForSequenceClassification.from_pretrained(trn_checkpoint)



Comparing the outputs on 3 samples

In [6]:
# def predict_sentiment(text, model, tokenizer, labels=None):
#     """
#     Predict sentiment for a given text using the specified model and tokenizer.

#     Args:
#         text (str): Input text to analyze
#         model: Hugging Face model
#         tokenizer: Corresponding tokenizer
#         labels (list, optional): Labels for prediction outputs. Defaults to ['negative', 'positive'].

#     Returns:
#         dict: Dictionary containing percentages for each label
#     """
#     if labels is None:
#         labels = ['negative', 'positive']

#     # Tokenize the input text
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

#     # Get model prediction
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Apply softmax to get probabilities
#     probs = F.softmax(outputs.logits, dim=-1)

#     # Convert to percentages
#     percentages = probs[0] * 100

#     # Return dictionary with labels and percentages
#     return {labels[i]: float(percentages[i]) for i in range(len(labels))}

# def compare_predictions(text):
#     """
#     Compare predictions between trained and untrained models.

#     Args:
#         text (str): Text to analyze

#     Returns:
#         tuple: Predictions from both models
#     """
#     untrn_pred = predict_sentiment(text, untrn_model, untrn_tokenizer)
#     trn_pred = predict_sentiment(text, trn_model, trn_tokenizer)

#     return {
#         "untrained_bert": untrn_pred,
#         "finetuned_distilbert": trn_pred
#     }

# # Test examples
# positive_example = "I love this movie! It's amazing."
# negative_example = "This was a terrible waste of time."
# ambiguous_example = "It was different than I expected."

# results = {
#     "positive": compare_predictions(positive_example),
#     "negative": compare_predictions(negative_example),
#     "ambiguous": compare_predictions(ambiguous_example)
# }

# # Print results in a readable format
# for sentiment_type, prediction in results.items():
#     print(f"\n--- {sentiment_type.upper()} EXAMPLE ---")
#     print(f"Text: {locals()[f'{sentiment_type}_example']}")
#     for model_name, scores in prediction.items():
#         print(f"\n  {model_name}:")
#         for label, score in scores.items():
#             print(f"    {label}: {score:.2f}%")

# Pre-processing the data

Instantiating the tokenizer

In [5]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Checking that the tokenizer works

In [8]:
input = tokenizer("This is a sample sentence.")
input

tokenizer.convert_ids_to_tokens(input["input_ids"])

['[CLS]', 'this', 'is', 'a', 'sample', 'sentence', '.', '[SEP]']

Tokenizing the dataset

In [6]:
def tokenize_function(example):
  return tokenizer(example['sentence'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Create smaller subsets for training and validation
small_train_dataset = tokenized_datasets["train"].select(range(600))
small_validation_dataset = tokenized_datasets["validation"].select(range(90))

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

//Test: Choosing 8 samples to check tensor sizes and clean-up unnecessary data

In [10]:
# # @title
# samples = tokenized_datasets["train"][:8]
# samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence"]}
# [len(x) for x in samples["input_ids"]]

Instantiating the collator

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

//Test: Creating a pre-processed batch with automatically added padding

In [12]:
# # @title
# batch = data_collator(samples)
# {k: v.shape for k, v in batch.items()}

# Training

Loading model && training args

In [8]:
from transformers import TrainingArguments, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments(
    "test-trainer",
    report_to=[],
    evaluation_strategy="epoch"
    )



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Defining the "compute_metrics" function

In [12]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    metric = evaluate.load("glue", "sst2")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

And the trainer..

In [13]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-13-9b90b4860e9e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.540869,0.866667
2,No log,0.879193,0.844444
3,No log,0.574621,0.866667


TrainOutput(global_step=225, training_loss=0.11126629299587673, metrics={'train_runtime': 12.4574, 'train_samples_per_second': 144.492, 'train_steps_per_second': 18.061, 'total_flos': 27326549765280.0, 'train_loss': 0.11126629299587673, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(small_validation_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)


In [ ]:
# @title
# # Test examples
from transformers import TextClassificationPipeline
import torch.nn.functional as F

# Create a text classification pipeline using your fine-tuned model
classifier = TextClassificationPipeline(
    model=trainer.model,
    tokenizer=tokenizer,
    return_all_scores=True
)

# Text examples
examples = [
    "I love this movie! It's amazing.",
    "This was a terrible waste of time.",
    "It was different than I expected."
]

# Get predictions for each example
for text in examples:
    # Get raw predictions
    raw_predictions = classifier(text)

    # Format and print results with percentages
    print(f"\nText: \"{text}\"")
    print("Predictions:")
    for result in raw_predictions[0]:
        label = result['label']
        score_percent = result['score'] * 100
        print(f"  {label}: {score_percent:.2f}%")

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)


In [34]:
import evaluate

metric = evaluate.load("glue", "sst2")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8555555555555555}

In [26]:
predictions

90